In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as ex
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df=pd.read_csv('../input/twitter-sentiment-dataset/Twitter_Data.csv')

In [44]:
df.head()

In [45]:
df.isnull().sum()

In [3]:
df.drop(df[df.isna().any(axis=1)].index,inplace=True)  #drop the rows is they are null

In [46]:
df['category'].value_counts()

In [47]:
ex.pie(df,df['category'],hole=0.6)

In [49]:
text = ''.join(df['clean_text'])

In [51]:
from wordcloud import WordCloud

In [52]:
#remove non english,numbers, "", hon'ble
def wc(text,label):

    wordcloud = WordCloud(width=500, height=500, background_color='black').generate(text)

    fig = plt.figure(figsize=(10,7), facecolor='k',edgecolor='k')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(label)
    plt.axis('off')
    #plt.show
wc(text,'full text')    

In [53]:
#now lets look wrt to 1,-1,0 tweets
pos = df[df['category']==1]
neg = df[df['category']==-1]
neutral = df[df['category']==0]

In [8]:
pos_text = "".join(pos['clean_text'])
wordcloud = WordCloud(width=500, height=500, background_color='black').generate(text)
fig = plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('positive tweets')
plt.axis('off')

In [9]:
neg_text = "".join(pos['clean_text'])
wordcloud = WordCloud(width=500, height=500, background_color='black').generate(text)
fig = plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('negative tweets')
plt.axis('off')

In [10]:
neg_text = "".join(pos['clean_text'])
wordcloud = WordCloud(width=500, height=500, background_color='black').generate(text)
fig = plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('neutral tweets')
plt.axis('off')

In [ ]:
#clean data

#remove numbers, emojis, non english, stopwords, ""  will include narendra modi, india, congress,rahul gandhi since they are ubiquitous

In [4]:
import emoji,re

def clean_text(text_str):
    text_str = emoji.get_emoji_regexp().sub('',text_str)#remove emojis
    text_str = re.sub('[^a-z-A-Z]',' ',text_str) #remove non english words
    text_str = re.sub('\s+',' ',text_str)  # remove extra space
    #text_str = re.sub('[^ws]', '',text_str)   #remove special characters
    return text_str

In [5]:
df['clean_1'] =df['clean_text'].apply(lambda x: clean_text(x))

In [6]:
import nltk
from nltk.corpus import stopwords

stp_words = stopwords.words('english')+['narendra modi','rahul gandhi','india','congress']+['modi','narendra']
def clean_text2(text_str):
    
    token=nltk.word_tokenize(text_str)#tokenize and lower easch word
    token=[t.lower() for t in token if t not in stp_words]
    return ' '.join(token)
df['clean_2']=df['clean_1'].apply(lambda x: clean_text2(x))




In [54]:
#now lets look wrt to 1,-1,0 tweets
pos_clean = df[df['category']==1]
neg_clean = df[df['category']==-1]
neutral_clean = df[df['category']==0]


fig,ax=plt.subplots(1,2,figsize=[30,10])
w1=WordCloud(background_color='white').generate(''.join(pos_clean['clean_2']))
ax[0].imshow(w1)
ax[0].axis('off')
ax[0].set_title('Positive Tweets',fontsize=40);

w2=WordCloud(background_color='white').generate(''.join(pos_clean['clean_2']))
ax[1].imshow(w2)
ax[1].axis('off')
ax[1].set_title('Negative Tweets',fontsize=40);

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [9]:

lb=LabelEncoder()#label encoding the target
df['category'] = lb.fit_transform(df['category'])

In [10]:
x_train,x_test,y_train,y_test = train_test_split(df['clean_2'],df['category'],train_size =0.75)

In [11]:
tfidf=TfidfVectorizer(analyzer='word',token_pattern=r'\w{2,}',ngram_range=(1,3),min_df=3)
train_tfidf = tfidf.fit_transform(x_train)
test_tfidf = tfidf.transform(x_test)

nb = MultinomialNB().fit(train_tfidf, y_train)
pred=nb.predict(test_tfidf)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

In [12]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [16]:
#parametres are taken after optuna 

parms = {'n_estimators': 462, 'max_depth': 3, 'learning_rate': 0.4647774206746099, 
         'reg_lambda': 0.28837849591744313, 'reg_alpha': 1.013304901229937, 'colsample_bytree': 0.10175699890042417, 'num_leaves': 66,
         'min_child_samples': 135, 'min_child_weight': 0.19486486818205837} 

lgbm = LGBMClassifier(objective = 'multi_crossentropy',device='gpu',**parms)
lgbm.fit(train_tfidf, y_train)
pred=lgbm.predict(test_tfidf)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

In [20]:
params ={'n_estimators': 592, 'max_depth': 4, 'learning_rate': 0.761222044751111, 'gamma': 0.0005027108090717986, 
         'reg_lambda': 1.5514539866514172, 'reg_alpha': 1.7864220561399082, 'colsample_bytree': 0.03909842319236968}

xgb = XGBClassifier(device='gpu',**parms)
xgb.fit(train_tfidf, y_train)
pred=xgb.predict(test_tfidf)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

# #Approach B using LSTM****

In [34]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from sklearn.model_selection import train_test_split 

In [31]:
max_length=100
vocab_size=5000
embedding_dim=64
trunc_type="post"
oov_tok="<OOV>"
padding_type="post"

In [32]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(df['clean_2'])

word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(df['clean_2'])
X = pad_sequences(X, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [35]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(256, dropout=0.2)))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())


In [36]:
#Splitting the data into training and testing
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

y=pd.get_dummies(df['category'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 22)


batch_size=64
history=model.fit(X_train,y_train, epochs=15, batch_size=batch_size,verbose = 1,
                  callbacks = [EarlyStopping(monitor='val_accuracy', patience=4)],validation_data=(X_test,y_test))

In [37]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [39]:
y_pred = model.predict(X_test)